In [ ]:
import glob
import os
import warnings
warnings.filterwarnings("ignore")

import cftime
import dask
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import pandas as pd
import xarray as xr

from esem import gp_model
from scipy import stats
import gpflow

from SALib.sample import fast_sampler
from SALib.analyze import fast

In [ ]:
### import some analysis functions we wrote for this project
import sys ; sys.path.append("..")
from ppe_analysis.analysis import *

In [ ]:
# Setup your PBSCluster
import dask
from dask.distributed import Client
from dask_jobqueue import PBSCluster

ncores=1
nmem='25GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    project='P93300641', # Input your project ID here
    walltime='03:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(30)

# Setup your client
client = Client(cluster)

In [ ]:
client.cluster

In [ ]:
def get_files(htape,yr0=1850,yr1=2014):
    d='/glade/campaign/asp/djk2120/PPEn11/transient/hist/'

    #find all files
    fs   = np.array(sorted(glob.glob(d+'*'+htape+'*')))
    yrs  = np.array([int(f.split(htape)[1][1:5]) for f in fs])
    keys = np.array([f.split('.clm2')[0][-7:] for f in fs])
    
    #bump back yr0, if needed
    uyrs=np.unique(yrs)
    yr0=uyrs[(uyrs/yr0)<=1][-1]
    
    #find index to subset files
    ix   = (yrs>=yr0)&(yrs<=yr1)
    yrs  = yrs[ix]
    keys = keys[ix]

    #subset and reshape files
    ny=np.sum(keys=='LHC0000')
    nens = int(len(keys)/ny)
    files = fs[ix].reshape([nens,ny])

    #convert to list of lists
    files = [list(f) for f in files]
    
    return files,np.unique(keys)

In [ ]:
def add_params(ds,df,keys):
    mems=df['member'].values
    ix1=0*mems==1
    for key in keys:
        ix1=(ix1)|(mems==key)

    nens=len(ds.ens)    
    ix2=0*np.arange(nens)==1
    for mem in mems:
        ix2=(ix2)|(ds.key==mem)


    params=[]    
    for p in df.keys():
        if p!='member':
            x=xr.DataArray(np.nan+np.zeros(nens),dims='ens')
            x[ix2]=df[p][ix1]
            ds[p]=x
            params.append(p)
    ds['params']=xr.DataArray(params,dims='param')

In [ ]:
def get_ds(dvs,htape,yr0=1850,yr1=2014,dropdef=False):
    
    def preprocess(ds):
        return ds[dvs]
    
    #read in the data
    files,keys = get_files(htape,yr0,yr1)
    if dropdef:
        files = files[1:]
        keys  = keys[1:]
    
    ds = xr.open_mfdataset(files,combine='nested',concat_dim=['ens','time'],
                       parallel=True,preprocess=preprocess)
    
    #fix the time dimension, if needed
    yr0=str(ds['time.year'].values[0])
    if (htape=='h0')|(htape=='h1'):
        ds['time']=xr.cftime_range(yr0,periods=len(ds.time),freq='MS',calendar='noleap')
    
    #add some param info, etc.
    df=pd.read_csv('/glade/campaign/asp/djk2120/PPEn11/csvs/lhc220926.txt')
    ds['key']=xr.DataArray(keys,dims=['ens'])
    add_params(ds,df,keys)
    
    #add landarea info
    la_file = '/glade/u/home/djk2120/clm5ppe/pyth/sparsegrid_landarea.nc'
    la = xr.open_dataset(la_file).landarea  #km2
    ds['la'] = la
    
    #add some extra variables, e.g. lat/lon
    tmp = xr.open_dataset(files[0][0])
    for v in tmp.data_vars:
        if 'time' not in tmp[v].dims:
            if v not in ds:
                ds[v]=tmp[v]
                
    if htape=='h1':
        ds['pft']=ds.pfts1d_itype_veg
    
    return ds

In [ ]:
def normalize(var):
    return (var-min(var))/(max(var)-min(var))

def unnormalize(norm_var,raw_var):
    return norm_var*np.array(max(raw_var)-min(raw_var)) + np.array(min(raw_var))

### Read in dataset

In [ ]:
dvs = ['TLAI','GPP','EFLX_LH_TOT','TWS','ER']
htape='h0'
ds=get_ds(dvs,htape,yr0=2005,yr1=2014)

In [ ]:
# get parameter sets 
lhckey = '/glade/campaign/asp/djk2120/PPEn11/csvs/lhc220926.txt'
df = pd.read_csv(lhckey)
ppe_params = df.drop(columns='member')
num_params = len(ppe_params.columns)

In [ ]:
# Read in satellite phenology run for reference
in_file = '/glade/scratch/linnia/LAI_SP_ctsm51d115/run/LAI_SP_ctsm51d115.clm2.h0.2000-02-01-00000.nc'
SP = xr.open_dataset(in_file)

f='/glade/campaign/asp/djk2120/PPEn11/CTL2010/hist/PPEn08_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'
ds2=xr.open_dataset(f)
ivals=ds2.grid1d_ixy.astype(int)-1  #python indexing starts at 0
jvals=ds2.grid1d_jxy.astype(int)-1
ds_SP = SP.TLAI[:,jvals,ivals]

### Global mean annual mean GPP

In [ ]:
# select target variable and divide LHC dataset into training and testing subsets
var_raw=gmean(amean(ds.GPP).mean(dim='year'),ds.la)*24*60*60 #Global avg GPP (kgC/m2/day)
var = normalize(var_raw)
n_test = 50 # number of ensemble members to test emulator
Y = var[1:].values # target variable excluding default model [0]
default = var[0].values # default model value

X_test, X_train = ppe_params[:n_test], ppe_params[n_test:]
y_test, y_train = Y[:n_test], Y[n_test:]

In [ ]:
# define kernal
kernel_linear = gpflow.kernels.Linear(active_dims=range(num_params),variance=1)
kernel_matern32 = gpflow.kernels.Matern32(active_dims=range(num_params), variance=1, lengthscales = np.tile(1,32))
kernel_matern52 = gpflow.kernels.Matern52(active_dims=range(num_params),variance=1,lengthscales=np.tile(1,32))
kernel_RBF = gpflow.kernels.RBF(active_dims = range(num_params), lengthscales=np.tile(1,num_params))

kernel = kernel_linear + kernel_matern32

# define emulator model and train
emulator = gp_model(np.array(X_train),np.array(y_train),kernel = kernel)
emulator.train()

In [ ]:
# Predict test points with emulator
y_pred, y_pred_var = emulator.predict(X_test.values)

# plot predicted values
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(y_test, y_pred, squared=False)

y_test_raw = unnormalize(y_test,var_raw)

plt.scatter(y_test_raw,unnormalize(y_pred,var_raw))
plt.plot([min(y_test_raw),max(y_test_raw)],[min(y_test_raw),max(y_test_raw)],c='k',linestyle='--',label='1:1 line')
plt.text(min(y_test_raw),max(y_test_raw),['RMSE = ',np.round(rms,3)])
plt.xlabel('CLM global mean annual mean GPP (kgC/m2/day)')
plt.ylabel('emulated global mean annual mean GPP (kgC/m2/day)')
plt.legend(loc = 'lower right')

In [ ]:
# One-at-a-time sensitivity 
n=21
s = np.linspace(0,1,n)
unif = pd.concat([pd.DataFrame(np.tile(0.5,n))]*num_params,axis=1) # hold all parameters at median value
unif.columns = ppe_params.columns
#unif['leafcn'] = np.tile(0.8,n) # change individual parameter default setting for OAAT

In [ ]:
plt.figure(figsize=[16,14])
sample = unif
for i, p in enumerate(ppe_params.columns):
    
    sample[p] = s
    oaat, v = emulator.predict(sample)
    sample[p] = np.tile(0.5,n) # set column back to median
    
    ax=plt.subplot(7,5,i+1)
    ax.fill_between(s, oaat-3.0*v**0.5, oaat+3.0*v**0.5,color='peru',alpha=0.4) # shade three standard deviations
    ax.plot(s,oaat,c='k')
    plt.text(0,0.2,p)
    plt.plot([0,1],[default, default],'--',c='k')
    ax.set_ylim([0,1])
    if i == 15:
        plt.ylabel('Global mean annual mean GPP (normalized)',fontsize=12)
    
#plt.savefig('../figs/param_sens/OAAT_sensitivity_GM-AM_GPP.png',dpi=200)

In [ ]:
# fourier amplitude sensitivity test w/emulator
problem = {
    'names': ppe_params.columns,
    'num_vars': num_params,
    'bounds': [[0, 1]],
}

sample = fast_sampler.sample(problem, 1000, M=4, seed=None)
Y, _ = emulator.predict(sample)
FAST = fast.analyze(problem, Y, M=4, num_resamples=100, conf_level=0.95, print_to_console=False, seed=None)
gpp_sens = pd.DataFrame.from_dict(FAST)
gpp_sens.index = gpp_sens.names
df_sens = gpp_sens.sort_values(by=['S1'],ascending=False)

In [ ]:
plt.figure(num=None, figsize=(12, 6), dpi=100, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 12})

ax = plt.subplot(1,1,1)
ax.bar(df_sens.names,df_sens['ST'],color='lightgrey',label='interactions')
ax.bar(df_sens.names,df_sens['S1'],color='darkolivegreen',label='main effects')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(loc='upper right')
plt.ylabel('Proportion of total emulated variance (total=1)')
plt.title('Global mean annual mean GPP')
plt.tight_layout()
plt.savefig('../figs/param_sens/FAST_sensitivity_GM-AM_GPP.png',dpi=200)

### Global mean annual mean Respiration

In [ ]:
# select target variable and divide LHC dataset into training and testing subsets
var_raw=gmean(amean(ds.ER).mean(dim='year'),ds.la)*24*60*60 #Global avg RE (kgC/m2/day)
var = normalize(var_raw)
n_test = 50 # number of ensemble members to test emulator
Y = var[1:].values # target variable excluding default model [0]
default = var[0].values # default model value

X_test, X_train = ppe_params[:n_test], ppe_params[n_test:]
y_test, y_train = Y[:n_test], Y[n_test:]

In [ ]:
# define kernal
kernel_linear = gpflow.kernels.Linear(active_dims=range(num_params),variance=1)
kernel_matern32 = gpflow.kernels.Matern32(active_dims=range(num_params), variance=1, lengthscales = np.tile(1,32))
kernel_matern52 = gpflow.kernels.Matern52(active_dims=range(num_params),variance=1,lengthscales=np.tile(1,32))
kernel_RBF = gpflow.kernels.RBF(active_dims = range(num_params), lengthscales=np.tile(1,num_params))
#kernel_poly = gpflow.kernels.Polynomial(active_dims=range(num_params),variance=3,offset=0)

kernel = kernel_linear + kernel_matern32

# define emulator model and train
emulator = gp_model(np.array(X_train),np.array(y_train),kernel = kernel)
emulator.train()

In [ ]:
# Predict test points with emulator
y_pred, y_pred_var = emulator.predict(X_test.values)

# plot predicted values
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(y_test, y_pred, squared=False)

y_test_raw = unnormalize(y_test,var_raw)

plt.scatter(y_test_raw,unnormalize(y_pred,var_raw))
plt.plot([min(y_test_raw),max(y_test_raw)],[min(y_test_raw),max(y_test_raw)],c='k',linestyle='--',label='1:1 line')
plt.text(min(y_test_raw),max(y_test_raw),['RMSE = ',np.round(rms,4)])
plt.xlabel('CLM global mean annual mean ER (kgC/m2/day)')
plt.ylabel('emulated global mean annual mean ER (kgC/m2/day)')
plt.legend(loc='lower right')

In [ ]:
# One-at-a-time sensitivity 
n=21
s = np.linspace(0,1,n)
unif = pd.concat([pd.DataFrame(np.tile(0.5,n))]*num_params,axis=1) # hold all parameters at median value
unif.columns = ppe_params.columns
#unif['leafcn'] = np.tile(0.8,n) # change individual parameter default setting for OAAT

In [ ]:
plt.figure(figsize=[14,12])
sample = unif
for i, p in enumerate(ppe_params.columns):
    
    sample[p] = s
    oaat, v = emulator.predict(sample)
    sample[p] = np.tile(0.5,n) # set column back to median
    
    ax=plt.subplot(7,5,i+1)
    ax.fill_between(s, oaat-3.0*v**0.5, oaat+3.0*v**0.5,color='peru',alpha=0.4) # shade three standard deviations
    ax.plot(s,oaat,c='k')
    plt.text(0,0.2,p)
    plt.plot([0,1],[default, default],'--',c='k')
    ax.set_ylim([0,1])
    if i == 15:
        plt.ylabel('Global mean annual mean ER (normalized)',fontsize=12)
    
plt.savefig('../figs/param_sens/OAAT_sensitivity_GM-AM_ER.png',dpi=200)

In [ ]:
# fourier amplitude sensitivity test
problem = {
    'names': ppe_params.columns,
    'num_vars': num_params,
    'bounds': [[0, 1]],
}

sample = fast_sampler.sample(problem, 1000, M=4, seed=None)
Y, _ = emulator.predict(sample)
FAST = fast.analyze(problem, Y, M=4, num_resamples=100, conf_level=0.95, print_to_console=False, seed=None)
er_sens = pd.DataFrame.from_dict(FAST)
er_sens.index = er_sens.names
df_sens = er_sens.sort_values(by=['S1'],ascending=False)

In [ ]:
plt.figure(num=None, figsize=(12, 6), dpi=100, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 12})

ax = plt.subplot(1,1,1)
ax.bar(df_sens.names,df_sens['ST'],color='lightgrey',label='interactions')
ax.bar(df_sens.names,df_sens['S1'],color='darkolivegreen',label='main effects')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(loc='upper right')
plt.ylabel('Proportion of total emulated variance (total=1)')
plt.title('Global mean annual mean Ecosystem Respiration (ER)')
plt.tight_layout()
plt.savefig('../figs/param_sens/FAST_sensitivity_GM-AM_ER.png',dpi=200)

### Global mean annual mean Evapotranspiration (W/m2)

In [ ]:
# select target variable and divide LHC dataset into training and testing subsets
var_raw = gmean(amean(ds.EFLX_LH_TOT).mean(dim='year'),ds.la)
var = normalize(var_raw)
n_test = 50 # number of ensemble members to test emulator
Y = var[1:].values # target variable excluding default model [0]
default = var[0].values # default model value

X_test, X_train = ppe_params[:n_test], ppe_params[n_test:]
y_test, y_train = Y[:n_test], Y[n_test:]

In [ ]:
# define kernal
kernel_linear = gpflow.kernels.Linear(active_dims=range(num_params),variance=1)
kernel_matern32 = gpflow.kernels.Matern32(active_dims=range(num_params), variance=1, lengthscales = np.tile(1,32))
kernel_matern52 = gpflow.kernels.Matern52(active_dims=range(num_params),variance=1,lengthscales=np.tile(1,32))
kernel_RBF = gpflow.kernels.RBF(active_dims = range(num_params), lengthscales=np.tile(1,num_params))
#kernel_poly = gpflow.kernels.Polynomial(active_dims=range(num_params),variance=3,offset=0)

kernel = kernel_linear + kernel_matern32

# define emulator model and train
emulator = gp_model(np.array(X_train),np.array(y_train),kernel = kernel)
emulator.train()

In [ ]:
# Predict test points with emulator
y_pred, y_pred_var = emulator.predict(X_test.values)

# plot predicted values
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(y_test, y_pred, squared=False)

y_test_raw = unnormalize(y_test,var_raw)

plt.scatter(y_test_raw,unnormalize(y_pred,var_raw))
plt.plot([min(y_test_raw),max(y_test_raw)],[min(y_test_raw),max(y_test_raw)],c='k',linestyle='--',label='1:1 line')
plt.text(min(y_test_raw),max(y_test_raw),['RMSE = ',np.round(rms,4)])
plt.xlabel('CLM global mean annual mean ET (W/m2)')
plt.ylabel('emulated global mean annual mean ET (W/m2)')
plt.legend(loc='lower right')

In [ ]:
# One-at-a-time sensitivity
n=21
s = np.linspace(0,1,n)
unif = pd.concat([pd.DataFrame(np.tile(0.5,n))]*num_params,axis=1) # hold all parameters at median value
unif.columns = ppe_params.columns
#unif['leafcn'] = np.tile(0.8,n) # change individual parameter default setting for OAAT

In [ ]:
plt.figure(figsize=[14,12])
sample = unif
for i, p in enumerate(ppe_params.columns):
    
    sample[p] = s
    oaat, v = emulator.predict(sample)
    sample[p] = np.tile(0.5,n) # set column back to median
    
    ax=plt.subplot(7,5,i+1)
    ax.fill_between(s, oaat-3.0*v**0.5, oaat+3.0*v**0.5,color='peru',alpha=0.4) # shade three standard deviations
    ax.plot(s,oaat,c='k')
    plt.text(0,0.1,p)
    plt.plot([0,1],[default, default],'--',c='k')
    ax.set_ylim([0,1])
    if i == 15:
        plt.ylabel('Global mean annual mean ET (normalized)',fontsize=12)
    
plt.savefig('../figs/param_sens/OAAT_sensitivity_GM-AM_ET.png',dpi=200)

In [ ]:
# fourier amplitude sensitivity test
problem = {
    'names': ppe_params.columns,
    'num_vars': num_params,
    'bounds': [[0, 1]],
}

sample = fast_sampler.sample(problem, 1000, M=4, seed=None)
Y, _ = emulator.predict(sample)
FAST = fast.analyze(problem, Y, M=4, num_resamples=100, conf_level=0.95, print_to_console=False, seed=None)
et_sens = pd.DataFrame.from_dict(FAST)
et_sens.index = et_sens.names
df_sens = et_sens.sort_values(by=['S1'],ascending=False)

In [ ]:
plt.figure(num=None, figsize=(12, 6), dpi=100, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 12})

ax = plt.subplot(1,1,1)
ax.bar(df_sens.names,df_sens['ST'],color='lightgrey',label='interactions')
ax.bar(df_sens.names,df_sens['S1'],color='darkolivegreen',label='main effects')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(loc='upper right')
plt.ylabel('Proportion of total emulated variance (total=1)')
plt.title('Global mean annual mean Evapotranspiration (EFLX_LH_TOT)')
plt.tight_layout()
plt.savefig('../figs/param_sens/FAST_sensitivity_GM-AM_ET.png',dpi=200)

### Global mean annual mean total soil water

In [ ]:
# select target variable and divide LHC dataset into training and testing subsets
var_raw = gmean(amean(ds.TWS).mean(dim='year'),ds.la)
var = normalize(var_raw)
n_test = 50 # number of ensemble members to test emulator
Y = var[1:].values # target variable excluding default model [0]
default = var[0].values # default model value

X_test, X_train = ppe_params[:n_test], ppe_params[n_test:]
y_test, y_train = Y[:n_test], Y[n_test:]

In [ ]:
# define kernal
kernel_linear = gpflow.kernels.Linear(active_dims=range(num_params),variance=1)
kernel_matern32 = gpflow.kernels.Matern32(active_dims=range(num_params), variance=1, lengthscales = np.tile(1,32))
kernel_matern52 = gpflow.kernels.Matern52(active_dims=range(num_params),variance=1,lengthscales=np.tile(1,32))
kernel_RBF = gpflow.kernels.RBF(active_dims = range(num_params), lengthscales=np.tile(1,num_params))
#kernel_poly = gpflow.kernels.Polynomial(active_dims=range(num_params),variance=3,offset=0)

kernel = kernel_linear + kernel_matern32

# define emulator model and train
emulator = gp_model(np.array(X_train),np.array(y_train),kernel = kernel)
emulator.train()

In [ ]:
# Predict test points with emulator
y_pred, y_pred_var = emulator.predict(X_test.values)

# plot predicted values
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(y_test, y_pred, squared=False)

y_test_raw = unnormalize(y_test,var_raw)

plt.scatter(y_test_raw,unnormalize(y_pred,var_raw))
plt.plot([min(y_test_raw),max(y_test_raw)],[min(y_test_raw),max(y_test_raw)],c='k',linestyle='--',label='1:1 line')
plt.text(min(y_test_raw),max(y_test_raw),['RMSE = ',np.round(rms,4)])
plt.xlabel('CLM global mean annual mean TWS (mm)')
plt.ylabel('emulated global mean annual mean TWS (mm)')
plt.legend(loc='lower right')

In [ ]:
# One-at-a-time sensitivity
n=21
s = np.linspace(0,1,n)
unif = pd.concat([pd.DataFrame(np.tile(0.5,n))]*num_params,axis=1) # hold all parameters at median value
unif.columns = ppe_params.columns
#unif['leafcn'] = np.tile(0.8,n) # change individual parameter default setting for OAAT

In [ ]:
plt.figure(figsize=[14,12])
sample = unif
for i, p in enumerate(ppe_params.columns):
    
    sample[p] = s
    oaat, v = emulator.predict(sample)
    sample[p] = np.tile(0.5,n) # set column back to median
    
    ax=plt.subplot(7,5,i+1)
    ax.fill_between(s, oaat-3.0*v**0.5, oaat+3.0*v**0.5,color='peru',alpha=0.4) # shade three standard deviations
    ax.plot(s,oaat,c='k')
    plt.text(0,0.36,p)
    plt.plot([0,1],[default, default],'--',c='k')
    ax.set_ylim([0,1])
    if i == 15:
        plt.ylabel('Global mean annual mean LAI (normalized)',fontsize=12)
    
plt.savefig('../figs/param_sens/OAAT_sensitivity_GM-AM_TWS.png',dpi=200)

In [ ]:
# fourier amplitude sensitivity test
problem = {
    'names': ppe_params.columns,
    'num_vars': num_params,
    'bounds': [[0, 1]],
}

sample = fast_sampler.sample(problem, 1000, M=4, seed=None)
Y, _ = emulator.predict(sample)
FAST = fast.analyze(problem, Y, M=4, num_resamples=100, conf_level=0.95, print_to_console=False, seed=None)
tws_sens = pd.DataFrame.from_dict(FAST)
tws_sens.index = tws_sens.names
df_sens = tws_sens.sort_values(by=['S1'],ascending=False)

In [ ]:
plt.figure(num=None, figsize=(12, 6), dpi=100, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 12})

ax = plt.subplot(1,1,1)
ax.bar(df_sens.names,df_sens['ST'],color='lightgrey',label='interactions')
ax.bar(df_sens.names,df_sens['S1'],color='darkolivegreen',label='main effects')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(loc='upper right')
plt.ylabel('Proportion of total emulated variance (total=1)')
plt.title('Global mean annual mean Total Water Storage (TWS)')
plt.tight_layout()
plt.savefig('../figs/param_sens/FAST_sensitivity_GM-AM_TWS.png',dpi=200)

### Global mean annual mean LAI

In [ ]:
# select target variable and divide LHC dataset into training and testing subsets
var_raw = gmean(amean(ds.TLAI).mean(dim='year'),ds.la)
var = normalize(var_raw)
n_test = 50 # number of ensemble members to test emulator
Y = var[1:].values # target variable excluding default model [0]
default = var[0].values # default model value

X_test, X_train = ppe_params[:n_test], ppe_params[n_test:]
y_test, y_train = Y[:n_test], Y[n_test:]

In [ ]:
sp_raw = gmean(amean(ds_SP).mean(dim='year'),ds.la)

In [ ]:
# define kernal
kernel_linear = gpflow.kernels.Linear(active_dims=range(num_params),variance=1)
kernel_matern32 = gpflow.kernels.Matern32(active_dims=range(num_params), variance=1, lengthscales = np.tile(1,32))
kernel_matern52 = gpflow.kernels.Matern52(active_dims=range(num_params),variance=1,lengthscales=np.tile(1,32))
kernel_RBF = gpflow.kernels.RBF(active_dims = range(num_params), lengthscales=np.tile(1,num_params))
#kernel_poly = gpflow.kernels.Polynomial(active_dims=range(num_params),variance=3,offset=0)

kernel = kernel_linear + kernel_matern32

# define emulator model and train
emulator = gp_model(np.array(X_train),np.array(y_train),kernel = kernel)
emulator.train()

In [ ]:
# Predict test points with emulator
y_pred, y_pred_var = emulator.predict(X_test.values)

# plot predicted values
rms = mean_squared_error(y_test, y_pred, squared=False)

y_test_raw = unnormalize(y_test,var_raw)

plt.scatter(y_test_raw,unnormalize(y_pred,var_raw))
plt.plot([min(y_test_raw),max(y_test_raw)],[min(y_test_raw),max(y_test_raw)],c='k',linestyle='--',label='1:1 line')
plt.text(min(y_test_raw),max(y_test_raw),['RMSE=',np.round(rms,4)])
plt.xlabel('CLM global mean annual mean LAI')
plt.ylabel('Emulated global mean annual mean LAI')
plt.legend(loc='lower right')

In [ ]:
# One-at-a-time sensitivity
n=21
s = np.linspace(0,1,n)
unif = pd.concat([pd.DataFrame(np.tile(0.5,n))]*num_params,axis=1) # hold all parameters at median value
unif.columns = ppe_params.columns
#unif['leafcn'] = np.tile(0.8,n) # change individual parameter default setting for OAAT
#unif['leaf_long'] = np.tile(0.15,n) # change individual parameter default setting for OAAT

In [ ]:
plt.figure(figsize=[16,14])
sample = unif
for i, p in enumerate(ppe_params.columns):
    
    sample[p] = s
    oaat, v = emulator.predict(sample)
    sample[p] = np.tile(0.5,n) # set column back to median
    
    ax=plt.subplot(7,5,i+1)
    ax.fill_between(s, oaat-3.0*v**0.5, oaat+3.0*v**0.5,color='peru',alpha=0.4) # shade three standard deviations
    ax.plot(s,oaat,c='k')
    ax.plot([0,1],[sp_raw,sp_raw],linestyle = '-.',c='green',label='CLM-SP')
    plt.text(0,0.02,p)
    plt.plot([0,1],[default, default],'--',c='k',label='default CLM')
    ax.set_ylim([0,1])
    if i == 15:
        plt.ylabel('Global mean annual mean LAI (normalized)',fontsize=12)
    if i == 0:
        plt.legend(loc = 'upper left')
    
plt.savefig('../figs/param_sens/OAAT_sensitivity_GM-AM_LAI.png',dpi=200)

In [ ]:
# fourier amplitude sensitivity test
problem = {
    'names': ppe_params.columns,
    'num_vars': num_params,
    'bounds': [[0, 1]],
}

sample = fast_sampler.sample(problem, 1000, M=4, seed=None)
Y, _ = emulator.predict(sample)
FAST = fast.analyze(problem, Y, M=4, num_resamples=100, conf_level=0.95, print_to_console=False, seed=None)
lai_sens = pd.DataFrame.from_dict(FAST)
lai_sens.index = lai_sens.names
df_sens = lai_sens.sort_values(by=['S1'],ascending=False)

In [ ]:
plt.figure(num=None, figsize=(12, 6), dpi=100, facecolor='w', edgecolor='k')
plt.subplot
plt.rcParams.update({'font.size': 12})

ax = plt.subplot(1,1,1)
ax.bar(df_sens.names,df_sens['ST'],color='lightgrey',label='interactions')
ax.bar(df_sens.names,df_sens['S1'],color='darkolivegreen',label='main effects')
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(loc='upper right')
plt.ylabel('Proportion of total emulated variance (total=1)')
plt.title('Global mean annual mean Leaf Area Index (TLAI)')
plt.tight_layout()
#plt.savefig('../figs/param_sens/FAST_sensitivity_GM-AM_TLAI.png',dpi=200)

### Heatmap

In [ ]:
# heatmap of all global mean variables
# group parameters by process
df_sens = pd.DataFrame({'Leaf Area':lai_sens['S1'],'Gross Primary Production':gpp_sens['S1'],'Ecosystem Respiration':er_sens['S1'],'Evapotranspiration':et_sens['S1'],'Total Water Storage':tws_sens['S1']})

params = ['jmaxb0', 'jmaxb1', 'leafcn', 'wc2wjb0', 'medlynintercept', 'medlynslope', 'kcha', 'tpu25ratio', 'tpuse_sf', 'theta_cj',
          'kmax', 'krmax', 'psi50', 'fstor2tran','crit_dayl', 'froot_leaf', 'nstem', 'slatop','leaf_long', 'stem_leaf', 'soilpsi_off',
          'grperc', 'lmr_intercept_atkin', 'lmrha', 'lmrhd', 'q10_mr', 'FUN_fracfixers', 'KCN', 'a_fix', 'fff', 'd_max', 'sucsat_sf']

df_sens = df_sens.reindex(params)
var_names = ['Leaf Area','Gross Primary Production','Ecosystem Respiration','Evapotranspiration','Total Water Storage']

In [ ]:
plt.figure(num=None, figsize=(20, 12), dpi=100, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 18})

ax = plt.subplot(1,1,1)
im = ax.imshow(100.0*df_sens.transpose(),interpolation = 'nearest',cmap='magma_r',vmin=0,vmax=25)


# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(params)), labels=params)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
ax.set_yticks(np.arange(len(var_names)), labels=var_names)

#cbar = plt.colorbar(im)
cbar = ax.figure.colorbar(im, fraction=0.015,pad = 0.05)
cbar.set_label("Relative influence \n on ensemble variance (%)")
plt.tight_layout()
plt.savefig('../figs/param_sens/FAST_sensitivity_GM-AM_heatmap.png',dpi=200)